In [ ]:
import os
import urllib.parse

import pandas as pd

In [ ]:
suspects = pd.read_parquet('../../data/interim/suspects_unique.parquet')

In [ ]:
suspects_acylcarnitines = suspects[suspects['CompoundName'].str.lower()
                                   .str.contains('carnitine', regex=False)]

In [ ]:
suspects_antibiotics = suspects[
    suspects['CompoundName'].str.lower()
    .str.contains('amoxicillin|azithromycin|nitrofurantoin')]

In [ ]:
suspects_apratoxin = suspects[suspects['CompoundName'].str.lower()
                              .str.contains('apratoxin', regex=False)]

In [ ]:
suspects_delta_saccharide = suspects[suspects['GroupDeltaMass'].between(162, 180)]

In [ ]:
def export(entries, filename):
    compound_names = (
        'Suspect related to ' + entries['CompoundName'] +
        ' with delta m/z ' + entries['GroupDeltaMass'].round(3).astype(str) +
        ' (putative explanation: ' + entries['Rationale'] +
        '; atomic difference: ' + entries['AtomicDifference'] + ')')
    with open(filename, 'w') as f_out:
        f_out.write(f'# Suspect spectral library '
                    f'{os.path.splitext(filename)[0]}\n\n')
        for library_usi, suspect_usi, name in zip(
                entries['LibraryUsi'], entries['SuspectUsi'], compound_names):
            f_out.write(f'### [{suspect_usi}]'
                        f'(https://metabolomics-usi.ucsd.edu/'
                        f'dashinterface/?'
                        f'usi1={urllib.parse.quote(library_usi)}&'
                        f'usi2={urllib.parse.quote(suspect_usi)}&'
                        f'max_intensity=150&cosine=shifted)\n\n')
            f_out.write(f'{name}\n\n')
            f_out.write(f'![](https://metabolomics-usi.ucsd.edu/png/mirror?'
                        f'usi1={urllib.parse.quote(library_usi)}&'
                        f'usi2={urllib.parse.quote(suspect_usi)}&'
                        f'max_intensity=150&cosine=shifted'
                        f'&annotate_peaks=[true%2Ctrue])\n\n')

In [ ]:
export(suspects_acylcarnitines, 'acylcarnitines.md')
export(suspects_antibiotics, 'antibiotics.md')
export(suspects_apratoxin, 'apratoxin.md')
export(suspects_delta_saccharide, 'delta_saccharide.md')